# Spike and Burst Detection with Shapelet Learning
Minimal functional notebook compatible with nbformat 5.10.4.

In [ ]:
import pyabf
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.interpolate import interp1d

In [ ]:
# Load ABF file
file_path = 'bursting/cell89basal.abf'
abf = pyabf.ABF(file_path)
signal = np.concatenate([abf.setSweep(i) or abf.sweepY for i in range(abf.sweepCount)])
dt = 1.0 / abf.dataRate
time = np.arange(len(signal)) * dt
print(f'Loaded {file_path}, duration: {time[-1]:.2f}s')

In [ ]:
# Spike detection
threshold = -35
spike_indices, _ = find_peaks(signal, height=threshold)
spike_times = time[spike_indices]

In [ ]:
# Burst detection
burst_threshold = 0.3
isi = np.diff(spike_times)
bursts = []
current_burst = [spike_times[0]]
for i in range(1, len(isi)):
    if isi[i-1] < burst_threshold:
        current_burst.append(spike_times[i])
    else:
        if len(current_burst) > 1:
            bursts.append((current_burst[0], current_burst[-1]))
        current_burst = [spike_times[i]]
if len(current_burst) > 1:
    bursts.append((current_burst[0], current_burst[-1]))
print(f'Total bursts detected: {len(bursts)}')

In [ ]:
# Burst normalization functions
def normalize_y(segment):
    return (segment - np.mean(segment)) / np.std(segment)

def rescale_x(t_segment, segment, n_points=100):
    f = interp1d(np.linspace(0,1,len(segment)), segment)
    return f(np.linspace(0,1,n_points))

In [ ]:
# Extract normalized bursts
def extract_normalized_bursts(burst_list, signal, time, n_points=100):
    normalized_bursts = []
    for start, end in burst_list:
        mask = (time >= start) & (time <= end)
        segment = signal[mask]
        segment_rescaled = rescale_x(time[mask], segment, n_points)
        segment_normalized = normalize_y(segment_rescaled)
        normalized_bursts.append(segment_normalized)
    return normalized_bursts

square_bursts_norm = extract_normalized_bursts(bursts, signal, time)
print(f'Example normalized burst shape: {square_bursts_norm[0].shape}')